In [ ]:
from quantumsim.functions.ansatz import *
from quantumsim.functions.min_methods import *
from quantumsim.functions.funciones import *
from quantumsim.optimizacion_structure import *
from quantumsim.variational_quantum_eigensolver import *

from pennylane import numpy as np

# Molecula de $H^2$
En este jupyter se estudia un dimero de hidrogeno sin campo usando el variational quantum deflation para obtener todos los eigenvalues y eigenvectors del hamiltoniano.

Con estos, podemos calcular algunos observables usando la temperatura.

### Espectro energetico (sto-3g)

In [ ]:
symbols = ["H", "H"]
coordinates = np.array([0.0, 0.0, -0.6614, 0.0, 0.0, 0.6614], requires_grad=True)

#Parametros del hamiltoniano molecular
params = {
    'mapping': "jordan_wigner",
    'charge': 0, 
    'mult': 1,
    'basis': 'sto-3g',
    'method': 'dhf'
}

#Construccion de la clase
molecule = vqe_molecular(symbols, coordinates, params)
molecule.set_group_characteristics()

In [ ]:
ansatz_params = {
    "repetitions": 2,
    "base": "default.qubit",
    "backend": None,
    "interface": "autograd",
    "shots": 1000.0,
    "token": None,
}

# Seteo de hiperparametros del circuito
molecule.set_device( ansatz_params )
molecule.set_node( ansatz_params )

rep = molecule.qubits*ansatz_params["repetitions"]

In [ ]:
minimizate_params = {
    "type": "SLSQP",
    "maxiter": 100,
    "tol": 0.000001,
    "number": molecule.qubits*ansatz_params["repetitions"]}

# Ejecucion del optimizador para calcular los niveles 
# de energia

optimizer = scipy_optimizer(minimizate_params)
optimizer.set_constrains()

energy, optimum = optimizer.VQD(molecule.cost_function, molecule.overlap_cost_function, 16, molecule.qubits)

In [ ]:
minimizate_params = {
    "type": "SLSQP",
    "step_theta": 0.4,
    "step_x": 0.4,
    "maxiter": 100,
    "tol": 0.000001,
    "number": molecule.qubits*ansatz_params["repetitions"]}

optimizer2 = gradiend_optimizer(minimizate_params)
energy, optimum = optimizer2.VQD(molecule.cost_function, molecule.overlap_cost_function, 16, molecule.qubits)

In [ ]:
import matplotlib.pyplot as plt

exacto = [-1.1361891 , -0.52188355, -0.52188349, -0.47844693, -0.47844693,
        -0.4784469 , -0.40317875, -0.40317875, -0.12044625,  0.30767557,
         0.3076756 ,  0.44909644,  0.4490965 ,  0.58332568,  0.75597221,
         1.01609778]

fig, ax = plt.subplots(1, 2, figsize=(8,4),)
ax[0].set_xlabel('Indices', fontsize=10)
ax[0].set_ylabel('Energy (Hartree)', fontsize=10)

ax[0].plot([i+1 for i in range(16)], np.sort(energy)-1, "o--", label="VQD")
ax[0].plot([i+1 for i in range(16)], exacto, "o--", label="ED")
ax[0].legend(loc="upper left", fontsize='medium',frameon=True, edgecolor='k', ncol=1)

ax[1].set_xlabel('Indices', fontsize=10)
ax[1].set_ylabel('Error', fontsize=10)
for i in range(len(energy)):
    ax[1].plot([i+1 for i in range(16)], [np.abs(float(j)- exacto[k]) for k,j in enumerate( np.sort(energy) )], ".", ls="dashed")
ax[1].set_yscale("log")

fig.suptitle('Espectro energetico $H^2$', fontsize=12)
fig.tight_layout()
plt.show()

### Espectro energetico (6-31g)

In [ ]:
symbols = ["H", "H"]
coordinates = np.array([0.0, 0.0, -0.6614, 0.0, 0.0, 0.6614], requires_grad=True)

#Parametros del hamiltoniano molecular
params = {
    'mapping': "jordan_wigner",
    'charge': 0, 
    'mult': 1,
    'basis': '6-31g',
    'method': 'dhf'
}

#Construccion de la clase
molecule = vqe_molecular(symbols, coordinates, params)
molecule.set_group_characteristics()

In [ ]:
ansatz_params = {
    "repetitions": 2,
    "base": "lightning.qubit",
    "backend": None,
    "interface": "autograd",
    "shots": 1000.0,
    "token": None,
}

# Seteo de hiperparametros del circuito
molecule.set_device( ansatz_params )
molecule.set_node( ansatz_params )

rep = molecule.qubits*ansatz_params["repetitions"]

In [ ]:
minimizate_params = {
    "type": "SLSQP",
    "maxiter": 100,
    "tol": 0.000001,
    "number": molecule.qubits*ansatz_params["repetitions"]}

# Ejecucion del optimizador para calcular los niveles 
# de energia

optimizer = scipy_optimizer(minimizate_params)
optimizer.set_constrains()
energy, optimum = optimizer.VQD(molecule.cost_function, molecule.overlap_cost_function, 2**8, molecule.qubits)

In [ ]:
exacto = [-1.15015652, -0.90214058, -0.90214058, -0.73210246, -0.73210246,
       -0.73210246, -0.57496493, -0.54501365, -0.54501365, -0.4573917 ,
       -0.4573917 , -0.40656487, -0.40656487, -0.30155458, -0.30155458,
       -0.30155458, -0.29116099, -0.14216493, -0.14216493, -0.14216493,
       -0.14216493, -0.10837638, -0.02759726, -0.02759726,  0.0144269 ,
        0.11888473,  0.11888473,  0.16377376,  0.16377376,  0.16377376,
        0.16409712,  0.16409712,  0.22755196,  0.27390298,  0.27390298,
        0.27390298,  0.29873517,  0.29873517,  0.31199174,  0.3470265 ,
        0.3470265 ,  0.3470265 ,  0.4552315 ,  0.4552315 ,  0.4552315 ,
        0.4552315 ,  0.48030427,  0.48030427,  0.48803866,  0.48803866,
        0.48803866,  0.52317769,  0.52317769,  0.5777653 ,  0.5777653 ,
        0.64804208,  0.66256352,  0.66256352,  0.69633705,  0.75597218,
        0.79397114,  0.81107038,  0.81844135,  0.81844135,  0.83791976,
        0.83791976,  0.83791976,  0.86820405,  0.86820405,  0.86820405,
        0.9191889 ,  0.9191889 ,  0.9191889 ,  0.9191889 ,  0.95013255,
        0.95013255,  0.95885311,  1.02158711,  1.02158711,  1.02158711,
        1.02578057,  1.05037131,  1.05037131,  1.13121933,  1.13121933,
        1.13399032,  1.13399032,  1.15354555,  1.15354555,  1.15354555,
        1.15365866,  1.25297179,  1.25297179,  1.25297179,  1.29860265,
        1.29860265,  1.33260626,  1.33260626,  1.38236489,  1.40630099,
        1.40630099,  1.40630099,  1.40630099,  1.41290225,  1.41290225,
        1.41290225,  1.45164037,  1.45164037,  1.45164037,  1.45164037,
        1.45164037,  1.50612659,  1.51017633,  1.55128046,  1.55128046,
        1.64638101,  1.64638101,  1.64638101,  1.73997   ,  1.74167811,
        1.74167811,  1.74572592,  1.74572592,  1.84606241,  1.84606241,
        1.84606241,  1.85191412,  1.85191412,  1.91574849,  1.94018169,
        1.94018169,  1.94018169,  1.99664825,  1.99664825,  2.00870016,
        2.00870016,  2.03706308,  2.05418604,  2.06691605,  2.06691605,
        2.06691605,  2.10441228,  2.10441228,  2.32578779,  2.32578779,
        2.32841073,  2.32841073,  2.32841073,  2.32841073,  2.35419242,
        2.3903518 ,  2.3903518 ,  2.3903518 ,  2.42852852,  2.42852852,
        2.44884269,  2.50910562,  2.50910562,  2.50910562,  2.50910562,
        2.58092517,  2.58092517,  2.58092517,  2.58485309,  2.68602724,
        2.68602724,  2.68602724,  2.73587659,  2.73587659,  2.79240792,
        2.80601128,  2.80601128,  2.80688101,  2.82419289,  2.82419289,
        2.96995983,  2.96995983,  3.01558428,  3.06116047,  3.20609338,
        3.20609338,  3.20609338,  3.30590693,  3.30590693,  3.34177717,
        3.34177717,  3.34177717,  3.34177717,  3.41591048,  3.43773992,
        3.43773992,  3.47580605,  3.47580605,  3.47580605,  3.48808675,
        3.48808675,  3.52960214,  3.55970721,  3.55970721,  3.64811837,
        3.64811837,  3.6735423 ,  3.6735423 ,  3.6735423 ,  3.7229796 ,
        3.7229796 ,  4.0875669 ,  4.16250705,  4.16250705,  4.16756108,
        4.16756108,  4.16756108,  4.16756108,  4.28725791,  4.30713719,
        4.30713719,  4.44280202,  4.44280202,  4.54833099,  4.54833099,
        4.63963575,  4.63963575,  4.63963575,  4.68744431,  4.7613104 ,
        4.7613104 ,  4.7613104 ,  4.77432143,  4.89455627,  5.2457357 ,
        5.2457357 ,  5.3567051 ,  5.3567051 ,  5.50963969,  5.50963969,
        5.50963969,  5.55671226,  5.60502757,  5.60502757,  5.60502757,
        5.78581477,  6.35072137,  6.35072137,  6.61390269,  6.61390269,
        6.61390269,  6.61907485,  6.76895633,  6.82688661,  7.22161907,
        7.22161907,  8.38070061,  8.38070061,  8.41907344,  8.41907344,
       10.46427032]

In [ ]:
import matplotlib.pyplot as plt

#exacto = [-1.1361891 , -0.52188355, -0.52188349, -0.47844693, -0.47844693,
#        -0.4784469 , -0.40317875, -0.40317875, -0.12044625,  0.30767557,
#         0.3076756 ,  0.44909644,  0.4490965 ,  0.58332568,  0.75597221,
#         1.01609778]

fig, ax = plt.subplots(1, 2, figsize=(8,4),)
ax[0].set_xlabel('Indices', fontsize=10)
ax[0].set_ylabel('Energy (Hartree)', fontsize=10)

ax[0].plot([i for i in range(16)], np.sort(energy), "o--", label="VQD")
ax[0].plot([i for i in range(len(exacto))], exacto, "o--", label="ED")
ax[0].legend(loc="upper left", fontsize='medium',frameon=True, edgecolor='k', ncol=1)

ax[1].set_xlabel('Indices', fontsize=10)
ax[1].set_ylabel('Error', fontsize=10)
for i in range(len(energy)):
    ax[1].plot([i for i in range(16)], [np.abs(float(j)- exacto[k]) for k,j in enumerate( np.sort(energy) )], ".", ls="dashed")
ax[1].set_yscale("log")

fig.suptitle('Espectro energetico $H^2$', fontsize=12)
fig.tight_layout()
plt.show()